SQL tutorials with Python and Apache Spark

In [4]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [6]:
# FindSpark simplifies the process of using Apache Spark with Python
import findspark
findspark.init()

In [10]:
#Create a SparkSession
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName("SQL tutorials").getOrCreate()

Load the dataset

In [12]:
df1 =spark.read.csv("matches.csv", header=True, inferSchema=True)
df1.createOrReplaceTempView("matches")

In [13]:
df2 =spark.read.csv("seasonstats.csv", header=True, inferSchema=True)
df2.createOrReplaceTempView("stats")

In [16]:
df3 =spark.read.csv("teams.csv", header=True, inferSchema=True)
df3.createOrReplaceTempView("teams")

SQL Basics

In [18]:
#Viewing all columns of the data

result = spark.sql("""
   SELECT *
   FROM stats
   WHERE season='2020/2021'
   LIMIT 5
""")
result.show()

+---------+---------------+---+----+----+---------+-------------+------+----+---------------+----------+------+-------+-------+
|   season|          teams|win|draw|lost|goals_for|goals_against|points|shot|short_on_target|free_kicks|pk_for|y_cards|r_cards|
+---------+---------------+---+----+----+---------+-------------+------+----+---------------+----------+------+-------+-------+
|2020/2021|Manchester City| 27|   5|   6|       83|           32|    86| 453|            140|        23|     6|     49|      5|
|2020/2021| Manchester Utd| 21|  11|   6|       73|           44|    74| 514|            178|        16|     5|     71|      4|
|2020/2021|      Liverpool| 20|   9|   9|       68|           42|    69| 478|            134|        15|     6|     49|      6|
|2020/2021|        Chelsea| 19|  10|   9|       58|           36|    67| 381|            124|        15|     3|     48|      0|
|2020/2021| Leicester City| 20|   6|  12|       68|           50|    66| 546|            193|        16|

In [120]:
#Using basic SELECT, FROM, WHERE 

result = spark.sql("""
   SELECT season,home_team,home_goals,away_goals,away_team,attendance,venue
   FROM matches
   WHERE season='2019/2020'
   LIMIT 5
""")

result.show(n=result.count(), truncate=False)

+---------+--------------+----------+----------+---------------+----------+---------------------+
|season   |home_team     |home_goals|away_goals|away_team      |attendance|venue                |
+---------+--------------+----------+----------+---------------+----------+---------------------+
|2019/2020|Liverpool     |4         |1         |Norwich City   |53333     |Anfield              |
|2019/2020|West Ham      |0         |5         |Manchester City|59870     |London Stadium       |
|2019/2020|Burnley       |3         |0         |Southampton    |19784     |Turf Moor            |
|2019/2020|Watford       |0         |3         |Brighton       |20245     |Vicarage Road Stadium|
|2019/2020|Crystal Palace|0         |0         |Everton        |25151     |Selhurst Park        |
+---------+--------------+----------+----------+---------------+----------+---------------------+



In [128]:
#Extracting top-performing teams in each season effortlessly! 

result = spark.sql("""
    WITH top_teams AS (SELECT 
        season,teams,points,
        ROW_NUMBER() OVER (PARTITION BY SEASON ORDER BY points DESC) as position
    FROM stats
    ) 
    SELECT season,teams,points,position
    FROM top_teams
    WHERE position=1
""")
result.show()

+---------+---------------+------+--------+
|   season|          teams|points|position|
+---------+---------------+------+--------+
|2021/2022|Manchester City|    93|       1|
|2020/2021|Manchester City|    86|       1|
|2019/2020|      Liverpool|    99|       1|
|2022/2023|Manchester City|    89|       1|
|2023/2024|Manchester City|    91|       1|
+---------+---------------+------+--------+



In [130]:
#Differenciating between ROW_NUMBER, RANK AND DENSE_RANK
result = spark.sql("""
    WITH top_teams AS (
    SELECT 
        season,teams,points,
        ROW_NUMBER() OVER (PARTITION BY SEASON ORDER BY points DESC) as position
    FROM stats
    ) 
    SELECT season,teams,points,position
    FROM top_teams
    WHERE position<=5 and season='2019/2020'
""")
result.show()

+---------+---------------+------+--------+
|   season|          teams|points|position|
+---------+---------------+------+--------+
|2019/2020|      Liverpool|    99|       1|
|2019/2020|Manchester City|    81|       2|
|2019/2020| Manchester Utd|    66|       3|
|2019/2020|        Chelsea|    66|       4|
|2019/2020| Leicester City|    62|       5|
+---------+---------------+------+--------+



In [132]:
#Differenciating between ROW_NUMBER, RANK AND DENSE_RANK
result = spark.sql("""
    WITH top_teams AS (
    SELECT 
        season,teams,points,
        RANK() OVER (PARTITION BY SEASON ORDER BY points DESC) as position
    FROM stats
    )  
    SELECT season,teams,points,position
    FROM top_teams
    WHERE position<=5 and season='2019/2020'
""")
result.show()

+---------+---------------+------+--------+
|   season|          teams|points|position|
+---------+---------------+------+--------+
|2019/2020|      Liverpool|    99|       1|
|2019/2020|Manchester City|    81|       2|
|2019/2020| Manchester Utd|    66|       3|
|2019/2020|        Chelsea|    66|       3|
|2019/2020| Leicester City|    62|       5|
+---------+---------------+------+--------+



In [114]:
#Differenciating between ROW_NUMBER, RANK AND DENSE_RANK
result = spark.sql("""
    WITH top_teams AS (
    SELECT 
        season,teams,points,
        DENSE_RANK() OVER (PARTITION BY SEASON ORDER BY points DESC) as position
    FROM stats
    )
    SELECT season,teams,points,position
    FROM top_teams
    WHERE position<=5 and season='2019/2020'
""")
result.show()

+---------+---------------+------+--------+
|season   |teams          |points|position|
+---------+---------------+------+--------+
|2019/2020|Liverpool      |99    |1       |
|2019/2020|Manchester City|81    |2       |
|2019/2020|Manchester Utd |66    |3       |
|2019/2020|Chelsea        |66    |3       |
|2019/2020|Leicester City |62    |4       |
|2019/2020|Tottenham      |59    |5       |
|2019/2020|Wolves         |59    |5       |
+---------+---------------+------+--------+

